### **Домашняя работа по кастомизации plotly**

##### **Задание №1**

1. Создайте датафрейм содержащий сумму и количество продаж треков в разбивке по странам и континентам.

2. Создайте scatter plot показывающий соотношение суммы продаж и количества треков.

3. Добавьте разделение по цвету на основе континента. Цвет Азии должен быть жёлтым, цвет Европы синим, цвет Северной Америки красным.

4. Добавьте название диаграмме. Шрифт должен быть 30, цвет шрифта серый.

5. Измените шрифт тиков (значения осей) на 14, цвет на серый.


##### **Задание №2**

1. Создайте датафрейм содержащий количество треков в разбивке по названиям жанров.

2. Создайте столбчатую диаграмму. По оси X должны быть названия жанров, по оси Y - количество треков.

3. Кастомизируйте диаграмму так, чтобы столбики всех жанров были серого цвета. За исключением жанров Jazz, Rock и Alternative & Punk. Эти жанры должны выделяться цветом. Цвета можете выбирать сами.

4. Уберите легенду.

5. Добавьте название диаграмме. Название должно быть курсивным.

6. Фон диаграммы должен быть белым.

In [2]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import sqlalchemy as db
from sqlalchemy import engine, URL, text

In [12]:
from sqlalchemy.engine import URL  
connection_url = URL.create(
    drivername = "postgresql+psycopg2",
    username = 'postgres',
    password = 123,
    host = 'localhost',
    port = 5432
    
)

engine = db.create_engine(connection_url)
conn = engine.connect()

In [35]:
query = """
select 
	invoice.billing_country as country
	, case 
			when billing_country in ('Australia', 'India')
				then 'Asia'
			when billing_country in ('Argentina', 'Chile', 'Brazil')
				then 'South America'
			when billing_country in ('USA','Canada')
				then 'North America'
			else 'Europe'
	end as continent
	, sum(total) as total_sales
	, count(*) as total_tracks
from invoice
group by country
order by total_sales desc;
"""

In [37]:
df1 = pd.read_sql(query, conn)
df1

,country,continent,total_sales,total_tracks
0,USA,North America,523.06,91
1,Canada,North America,303.96,56
2,France,Europe,195.10,35
3,Brazil,South America,190.10,35
4,Germany,Europe,156.48,28
5,United Kingdom,Europe,112.86,21
6,Czech Republic,Europe,90.24,14
7,Portugal,Europe,77.24,14
8,India,Asia,75.26,13
9,Chile,South America,46.62,7


In [54]:

fig1 = px.scatter(
    df1,
    x='total_sales',
    y='total_tracks',
    color='continent',
    color_discrete_map={'Asia': 'yellow','Europe': 'blue','North America': 'red'
},
    hover_name='country',
    title='Соотношение суммы продаж и количества треков'
)

# Кастомизация заголовка
fig1.update_layout({
    'title':{
        'text':'Соотношение суммы продаж и количества треков',
        'font':{'size':30, 'color':'gray'}},
    'xaxis': {
        'title':'Сумма продаж',
        'tickfont':{'size':14, 'color':'gray'}},
    'yaxis':{
        'title':'Количество треков',
        'tickfont': {'size':14, 'color':'gray'}}
})

# Показываем график
fig1.show()

In [41]:
query2 = """ 
select g.name as genre, count(t.track_id) AS track_count
from genre g
join track t on g.genre_id = t.genre_id
group by g.name
order by track_count desc;
"""
df2 = pd.read_sql(query2, conn)
df2

,genre,track_count
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


In [72]:
fig2 = px.bar(
    df2,
    x='genre',
    y='track_count',
    color='genre',
    color_discrete_sequence = ['grey'],
    color_discrete_map = {'Jazz': 'green', 'Rock': 'red', 'Alternative & Punk':'blue'} 
)
fig2.update_layout({
    'showlegend': False,
    'title' : {'text':'<i>Количество треков по жанрам</i>', 'font': {'size': 30, 'color': 'black'}, 'x':.08},
    'xaxis': {
        'title':'Жанры',
        'tickfont':{'size':14, 'color':'black'}},
    'yaxis':{
        'title':'Количество треков',
        'tickfont': {'size':14, 'color':'black'}},
    'template': 'plotly_white'
})
fig2.show()

In [ ]:
conn.close()